# Initialization

## Importing necessary packages

In [1]:
import os
import datetime
import random
import pandas as pd
import numpy as np
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torchvision.transforms as transforms
import torchvision.models as models
import lightning as L
from sklearn.metrics import cohen_kappa_score

import yaml

from models.enet_models import ENetModel
from hyperparameters import HyperParameters
from lightning_module import LightningModel
from data_module import PandaDataModule

from typing import List, Tuple, Dict

## Parameters

In [ ]:
NUM_WORKERS = 4

## Input Data Source

In [19]:
TRAINING_DATA_FOLDER: str = 'train-closetiled-prostate-36x256x256'
TESTING_DATA_FOLDER: str = 'test-closetiled-prostate-36x256x256'
DATASET_FOLDER_PATH: str = os.path.join(os.path.abspath('..'), 'dataset')
TESTING_DATA_CSV_PATH: str = os.path.join(DATASET_FOLDER_PATH, TESTING_DATA_FOLDER, 'test.csv')

# Trained Model checkpoint

Setting the file paths for the trained model checkpoint and the hyperparameters `.yaml` file

In [11]:
chkpt_file_path: str = os.path.join(os.path.abspath('..'),
                                    'logs\\resnet50-20240423-000603\\fold_2\\epoch=26-kappa=0.7400.ckpt')
hparams_file_path: str = os.path.join(os.path.abspath('..'),
                                      'logs\\resnext50_semi-20240421-102016\\fold_1\\hparams.yaml')

# Loading the saved HyperParameters `.yaml` file
dict_h_params: Dict[str, any] = {}
with open(hparams_file_path, 'r') as stream:
    dict_yaml = yaml.safe_load(stream)
    dict_h_params = dict_yaml

# Instantiating a HyperParameters object
h_params = HyperParameters.load_from_dict(dict_h=dict_h_params)

# EfficientNet Model

In [ ]:
model = ENetModel(c_out=h_params.c_out,
                  backbone=h_params.backbone)

print(model.summary())

# Lightning Model

In [ ]:
lt_model = LightningModel.load_from_checkpoint(model=model,
                                               h_params=dict_h_params,
                                               checkpoint_path=chkpt_file_path,
                                               hparams_file=hparams_file_path,
                                               map_location=None)

# Data Module

In [ ]:
data_module: L.LightningDataModule = PandaDataModule(train_folder_path=os.path.join(DATASET_FOLDER_PATH, TRAINING_DATA_FOLDER),
                                                     test_folder_path=os.path.join(DATASET_FOLDER_PATH, TESTING_DATA_FOLDER),
                                                     # list_train_idx=list_train_idx,
                                                     # list_val_idx=list_val_idx,
                                                     num_workers=NUM_WORKERS,
                                                     h_params=h_params)

# Model Predictions

## Predictions on the complete Hold-out Test set

In [ ]:
# Initializing the Lighning Trainer for running Predictions
trainer: L.Trainer = L.Trainer(max_epochs=h_params.num_epochs,
                               accelerator="gpu" if torch.cuda.is_available() else "cpu",
                               devices=1,
                               num_nodes=1,
                               accumulate_grad_batches=1)

# Testing the model
testing_results = trainer.predict(model=lt_model,
                                  datamodule=data_module)

## Prediction on a single image

In [27]:
# Load the `test.csv` file from the Hold-out test set
df_testing_data : pd.DataFrame = pd.read_csv(TESTING_DATA_CSV_PATH)

df_testing_data

,image_id,data_provider,isup_grade,gleason_score
0,332db6f0f6f290e521b714b4d209ff25,karolinska,2,3+4
1,9a3ad5844e796a97e725de8fa7518144,radboud,2,3+4
2,be9a4ad52349a7a82f4e8ac2154e93df,karolinska,3,4+3
3,345f3e86a3dcc26dce73ceeacc5a4a0a,radboud,3,4+3
4,6de4998fbdf6ebd012fd6b727890e582,karolinska,1,3+3
...,...,...,...,...
990,e9c567814f3d582607f9eff2a3b7668c,karolinska,0,0+0
991,a4119823dddda4b7cd9be59893173b0a,karolinska,0,0+0
992,9b16f18bbaecf31ae19380009a076862,radboud,5,4+5
993,af48aa2477b1f92d2b98a6eb6993873f,karolinska,1,3+3


In [42]:
random_idx: int = random.randint(0, len(df_testing_data))
df_row: pd.Series = df_testing_data.iloc[random_idx]

df_row.data_provider

# Displaying the original testing image (.tiff)


# Creating image tiles from the input testing image


# Create a big image from the tiles


# Take the model's output for the input image


# Calculate scores comparing the predicted and target outputs



'karolinska'